In [2]:
# Dependencias
import pickle
import tensorflow as tf
import cv2
import mediapipe as mp
import numpy as np

In [1]:
# Cargamos el modelo
model = tf.keras.models.load_model('./model/cnn_model.h5')
labels_dict = {0: 'Peace', 1: 'Like'} # Labels para las clases

NameError: name 'tf' is not defined

In [3]:
def app():
    # Definimos la captura de video
    cap = cv2.VideoCapture(0)

    # Definimos el módulo de manos de MediaPipe
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles

    hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5, max_num_hands=2)

    while True:
        # Lista para almacenar características para cada mano individual
        data_aux_list = [] 

        # Leemos un fotograma de la cámara
        ret, frame = cap.read()

        # Obtenemos las dimensiones del fotograma
        H, W, _ = frame.shape

        # Convertir el fotograma a RGB para el procesamiento con MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Procesar el fotograma con MediaPipe
        results = hands.process(frame_rgb)

        # Si hay manos detectadas
        if results.multi_hand_landmarks:
            # Para cada mano detectada
            for hand_landmarks in results.multi_hand_landmarks:
                # Lista para guardar las coordenadas X y Y de cada landmark
                x_ = []
                y_ = []

                # Dibujar landmarks
                mp_drawing.draw_landmarks(
                    frame,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style()
                )

                # Extraemos la coordenada en X y Y de esta y las guardamos en x_ y_
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y

                    x_.append(x)
                    y_.append(y)

                # Lista para guardar las coordenadas normalizadas
                data_aux = []

                # Normalizar y almacenar las coordenadas en data_aux
                for i in range(len(hand_landmarks.landmark)):
                    x = hand_landmarks.landmark[i].x
                    y = hand_landmarks.landmark[i].y
                    data_aux.append(x - min(x_))
                    data_aux.append(y - min(y_))

                # Coordenadas del cuadro delimitador
                x1 = int(min(x_) * W) - 10
                y1 = int(min(y_) * H) - 10

                x2 = int(max(x_) * W) - 10
                y2 = int(max(y_) * H) - 10

                # Realizar la predicción
                data_aux_reshaped = np.asarray(data_aux).reshape(-1, 6, 7, 1)
                data_aux_reshaped = data_aux_reshaped.astype(np.float32)
                prediction = model.predict(data_aux_reshaped)
                predicted_class_index = np.argmax(prediction)
                # Obtener el nombre de la clase predicha
                predicted_character = labels_dict[predicted_class_index]


                # En data_aux_list guardamos las coordenadas normalizadas, las coordenadas del cuadro delimitador y el nombre la clase
                data_aux_list.append((data_aux, (x1, y1, x2, y2, predicted_character)))

        # Iterar a traves de cada mano y mostrar los resultados utilizando los valores almacenados
        for data_aux, (x1, y1, x2, y2, predicted_character) in data_aux_list:
            # Mostrar el resultado para cada mano utilizando las coordenadas del cuadro delimitador y el carácter predicho
            cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3, cv2.LINE_AA)
        cv2.putText(frame, 'Press "Esc" to exit', (0, frame.shape[0] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
        # Mostrar el fotograma resultante
        cv2.imshow('frame', frame)

        # ESC para salir
        key = cv2.waitKey(1)
        if key == 27:
            break

    # Liberar la cámara y cerrar todas las ventanas
    cap.release()
    cv2.destroyAllWindows()


In [4]:
# Llamar a la función principal
app()

1/1 [==============================] - 0s 198ms/step


UnboundLocalError: local variable 'predicted_character' referenced before assignment

: 